In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

llm.invoke("who am i")



AIMessage(content="Unfortunately, I'm a large language model, I don't have the ability to know your personal identity or secrets. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations.\n\nIf you're feeling uncertain about who you are or want to explore some questions about yourself, here are a few prompts that might be helpful:\n\n* What do you value most in life?\n* What are your goals and aspirations?\n* What makes you feel fulfilled and happy?\n* What are your strengths and weaknesses?\n\nFeel free to answer any or all of these questions, and I'll do my best to provide some insights or suggestions.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-22T07:46:54.289054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5103318833, 'load_duration': 2489081083, 'prompt_eval_count': 28, 'prompt_eval_duration': 350593291, 'eval_count': 133, 'eval_duration': 1596569414, 'logprobs'

In [3]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate(
    template="What is the capital of the {country}? Return the name of city only",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "korea"})
ai_message =  llm.invoke(prompt)

output_parser = StrOutputParser()
answer = output_parser.invoke(llm.invoke(prompt))



In [4]:
print(ai_message)

content='Seoul' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-22T07:46:54.42583Z', 'done': True, 'done_reason': 'stop', 'total_duration': 123468875, 'load_duration': 61761292, 'prompt_eval_count': 40, 'prompt_eval_duration': 21469084, 'eval_count': 3, 'eval_duration': 24807126, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'} id='lc_run--019b4506-86dd-7f72-b82d-f45c55ff2779-0' usage_metadata={'input_tokens': 40, 'output_tokens': 3, 'total_tokens': 43}


In [5]:
print(answer)

Seoul.


In [8]:
from locale import currency
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The  population the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

structured_llm = llm.with_structured_output(CountryDetail)

In [9]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_propmt = PromptTemplate(
    template="""Give following information about {country}:
    - Capit
    - Population
    - Language
    - Currency
    
    return it in JSON format. and return the JSON dictionary only
    """,
    input_variables=["country"]
)

json_ai_message = structured_llm.invoke(country_detail_propmt.invoke({"country": "korea"}))

In [28]:
(json_ai_message)

CountryDetail(capital='Seoul', population=51, language='Korean', currency='South Korean won')

In [29]:
json_ai_message.model_dump()

{'capital': 'Seoul',
 'population': 51,
 'language': 'Korean',
 'currency': 'South Korean won'}

In [16]:
capitail_chain = prompt_template | llm | output_parser

In [17]:
capitail_chain.invoke({"country": "korea"})

'Seoul'

In [18]:
country_propmt = PromptTemplate(
    template="""Guess the name of the country based on the following information:
    {information}
    Retrun the name of the country only
    """,
    input_variables=["information"]
)
country_chain = country_propmt | llm | output_parser
country_chain.invoke({"information": "This country is vert famous for its pasta"})

'Italy.'

In [20]:
from langchain_core.runnables import RunnablePassthrough

# final_chain = {"country":country_chain} | capitail_chain
# final_chain.invoke({"information": "This country is vert famous for its pasta"}) 
final_chain = {"information": RunnablePassthrough()}|{"country":country_chain} | capitail_chain
final_chain.invoke( "This country is vert famous for its pasta") 





'Rome.'